# INF161 Project - Modellering og prediksjon

- Input skal være klargjort data fra tidligere med eventuelle features.
- Output skal være en maskinlæringsmodell med en forventet RMSE (root mean squared error).

Skal bruke tidligere klartgjort data til å lage en maskinlæringsmodel for å predikere antall sykler for 2022. 
Målet er å lage 3 forskjellige modeller før den som passer best blir valgt. 

In [1]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.experimental import enable_iterative_imputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso

In [2]:
merged_df = pd.read_csv('clean_data/merged_df.csv')
target_df = pd.read_csv('clean_data/target_df.csv')

# Preprocess

In [3]:
# Ser at det er få nullverdier med tanke på størrelsen på datasettet. 
merged_df.isnull().sum()

År                  0
Måned               0
Dato                0
Ukedag              0
Klokkeslett         0
Lufttemperatur     62
Vindstyrke         61
Lufttrykk          61
Fellesferie         0
Globalstraling     53
Solskinstid        69
Volum             213
dtype: int64

 Fjerne/erstatte NaN-verdier:
Siden det er så få nullverdier i 'Volum' (213/40 748 rader), velger jeg bare å fjerne alle rader hvor det er NaN-verdier i 'Volum' fra hele datasettet.

Det er også få nullverdier i de andre kolonnene, men vil ta i bruk en strategi.
Vil erstatte NaN-verdier i feature kolonnene med en verdi basert på data i datasettet. Bruker sklearn sin SimpleImputer for å erstatte verdiene med en enkel strategi, mean. Den erstatter NaN-verdiene med med gjennomsnittsverdien i kolonnen. Dette gjøres i funksjonen imputer(). Har forsøkt å bruke 'median' og 'most_frequent' som strategi i tillegg, men resultatene i modellen blir tilnærmet lik.

Bruker Train test split på samme måte som i preperation.ipynb

 Feature engineering:
One-hot-encoding er en prosess hvor kategoriske variabler blir konvertert til en form som kan sørge for bedre resultat i prediksjon ved bruk av maskinlæringsalgoritmer.

Pandas get_dummies API kan bli brukt for å transformere kategoriske variabler til dummy variabler. Dette er en versjon av one-hot-encoding, og brukes nedenfor. Jeg bruker det i feature kolonnen 'Ukedag'.

!NB, gjør denne encodingen inne i app.py

In [4]:
#merged_df = pd.get_dummies(merged_data, columns = ['Ukedag'])

In [5]:
# Fjerner alle rader i merged_df hvor Volum er NaN
merged_df.dropna(subset=['Volum'], inplace = True)

In [6]:
# Lagrer feature matrisen som 'X'
feature_cols = ['År','Måned', 'Dato','Ukedag', 'Klokkeslett', 'Lufttemperatur',
                'Lufttrykk','Globalstraling', 'Solskinstid', 'Fellesferie']

X = merged_df[feature_cols].copy()

# Lagrer responsvektoren som 'y' 
y = merged_df['Volum'].copy()

In [7]:
# Splitter slik at det først blir 70% treningsdata og 30% testdata
X_train, X_test1, y_train, y_test1 = train_test_split(X, y, test_size=0.3, shuffle = False)
    
# For at det skal bli 15% valideringsdata splittes testdataen opp i 50/50
X_test, X_val, y_test, y_val = train_test_split(X_test1, y_test1, test_size=0.5, shuffle = False)

In [8]:
# imputer funksjonen vil erstatte NaN-verdier i datasettene som tas inn og bruke SimpleImputer på ønskede kolonner.
def imputer(to_impute, feature_cols):
    for df in to_impute:
        cols = df.loc[:,feature_cols]
        imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean').fit(cols)
        df[feature_cols] = imp_mean.transform(cols)
    return df

feature_cols = ['År','Måned', 'Dato','Ukedag', 'Klokkeslett', 'Lufttemperatur',
                'Lufttrykk','Globalstraling', 'Solskinstid', 'Fellesferie']

to_impute = [X_train, X_test, X_val]
imputer(to_impute, feature_cols)

,År,Måned,Dato,Ukedag,Klokkeslett,Lufttemperatur,Lufttrykk,Globalstraling,Solskinstid,Fellesferie
34659,2019.0,7.0,1.0,1.0,19.0,9.966667,1001.133333,31.650000,0.5,1.0
34660,2019.0,7.0,1.0,1.0,20.0,9.983333,1002.216667,15.266667,0.0,1.0
34661,2019.0,7.0,1.0,1.0,21.0,9.500000,1003.033333,1.166667,0.0,1.0
34662,2019.0,7.0,1.0,1.0,22.0,9.233333,1003.566667,-1.150000,0.0,1.0
34663,2019.0,7.0,1.0,1.0,23.0,9.050000,1004.283333,-1.316667,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
40733,2020.0,3.0,10.0,2.0,20.0,4.550000,975.950000,-0.133333,0.0,0.0
40734,2020.0,3.0,10.0,2.0,21.0,5.233333,976.233333,-0.183333,0.0,0.0
40735,2020.0,3.0,10.0,2.0,22.0,5.733333,976.716667,0.000000,0.0,0.0
40736,2020.0,3.0,10.0,2.0,23.0,6.266667,977.300000,0.183333,0.0,0.0


In [9]:
# Sjekker for nullverdier
print(X_train.isnull().sum())
print(X_test.isnull().sum())
print(X_val.isnull().sum())
print(y_train.isnull().sum())
print(y_test.isnull().sum())
print(y_val.isnull().sum())

År                0
Måned             0
Dato              0
Ukedag            0
Klokkeslett       0
Lufttemperatur    0
Lufttrykk         0
Globalstraling    0
Solskinstid       0
Fellesferie       0
dtype: int64
År                0
Måned             0
Dato              0
Ukedag            0
Klokkeslett       0
Lufttemperatur    0
Lufttrykk         0
Globalstraling    0
Solskinstid       0
Fellesferie       0
dtype: int64
År                0
Måned             0
Dato              0
Ukedag            0
Klokkeslett       0
Lufttemperatur    0
Lufttrykk         0
Globalstraling    0
Solskinstid       0
Fellesferie       0
dtype: int64
0
0
0


In [10]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(28367, 10)
(6079, 10)
(6079, 10)


In [11]:
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(28367,)
(6079,)
(6079,)


In [12]:
# Target_df må være lik merged_df fordi modellen må kjøre på samme variabler. Gjentar prosessen.
target_df['Volum'] = target_df['Volum'].fillna(0)

In [13]:
# Bruker SimpleImputer på features i target_df, mean som strategi.
feature_cols = ['År','Måned', 'Dato','Ukedag', 'Klokkeslett', 'Lufttemperatur',
                'Lufttrykk','Globalstraling', 'Solskinstid', 'Fellesferie']

target_df = target_df[feature_cols].copy()
cols = target_df.loc[:,feature_cols]
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean').fit(cols)
target_df[feature_cols] = imp_mean.transform(cols)

# Dummy baseline model

Vil sammenligne en enkel baseline model med de faktiske modellene som skal brukes for å få en indikasjon
på om de er bra eller ikke. Bruker Scikit Learn sin DummyRegressor med mean som strategi. 

Kilde: https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyRegressor.html

In [14]:
from sklearn.dummy import DummyRegressor

# Lager en dummy regressor
dummy_reg = DummyRegressor(strategy='mean')

# Tilpasser på treningsdata
dummy_reg.fit(X_train, y_train)

# Lager prediksjoner
y_pred_train = dummy_reg.predict(X_train)
y_pred_val = dummy_reg.predict(X_val)

# Beregner root mean squared error for Y train set
RMSE_train = mean_squared_error(y_train, y_pred_train, squared = False)
print("RMSE on training data using DummyRegressor:", RMSE_train)

# Beregner root mean squared error for Y validation set
RMSE_val = mean_squared_error(y_val, y_pred_val, squared = False)
print("RMSE on validation data using DummyRegressor:", RMSE_val)


RMSE on training data using DummyRegressor: 72.79800462326035
RMSE on validation data using DummyRegressor: 71.71814501781552


# Kjører modellene

Forklaring av modeller:
- Lasso
    En type lineær regresjon som bruker "shrinkage" eller "krymping". Dataverdier blir krympet mot et sentralt
    punkt, som gjennomsnittet. Lasso-modellen oppmuntrer til enkle modeller, dvs modeller med få parametre.
- LinearRegression
    En lineær modell med koeffisienter w (w1, ..., wp) for å minimere "residual sum" av kvadrater mellom de
    obeserverte punktene i datasettet og dei predikerte punktene av den lineære tilnærmingen.
- K-Nearest Neighbour
    Enkel maskinlæringsalgoritme som antar likheten mellom den nye dataen og tilgjengelig data, som legger den nye
    dataen i den kategorien som ligner mest på de tilgjengelige. 
- RandomForestRegression
    "A random forest is a meta estimator that fits a number of classifying decision trees on various sub-samples of
    the dataset and uses averaging to improve the predictive accuracy and control over-fitting." 
    Kilde: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html   

In [15]:
import math

# modeller
la = Lasso()
lr = LinearRegression()
knn_5 = KNeighborsClassifier(n_neighbors = 5)
knn_10 = KNeighborsClassifier(n_neighbors = 10)
knn_15 = KNeighborsClassifier(n_neighbors = 15)
rfr_10 = RandomForestRegressor(max_depth=10)
rfr_15 = RandomForestRegressor(max_depth=15)
rfr_20 = RandomForestRegressor(max_depth=20)
rfr_25 = RandomForestRegressor(max_depth=25)

models = [la, lr, knn_5, knn_10, knn_15, rfr_10,rfr_15, rfr_20, rfr_25]

def run_model(model, X_train, X_val, y_train, y_val):
    
    # fit model
    model.fit(X_train, y_train)
    
    # prediksjon
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    
    # beregner root mean squared error
    RMSE_train = mean_squared_error(y_train, y_pred_train, squared = False)
    RMSE_val = mean_squared_error(y_val, y_pred_val, squared = False)
    
    print("\n")
    print(format(model))
    print("RMSE on validation data:", RMSE_val)
    print("RMSE on train data:", RMSE_train)
    
for model in models:
    run_model(model, X_train, X_val, y_train, y_val)



Lasso()
RMSE on validation data: 64.6173061953203
RMSE on train data: 63.741758097705855


LinearRegression()
RMSE on validation data: 64.96277126022237
RMSE on train data: 63.62149838432871


KNeighborsClassifier()
RMSE on validation data: 75.92920465208717
RMSE on train data: 71.49762752255764


KNeighborsClassifier(n_neighbors=10)
RMSE on validation data: 78.61203856494711
RMSE on train data: 76.36795058818996


KNeighborsClassifier(n_neighbors=15)
RMSE on validation data: 77.72951676133071
RMSE on train data: 76.10188208678491


RandomForestRegressor(max_depth=10)
RMSE on validation data: 24.809541177104858
RMSE on train data: 22.762171792648875


RandomForestRegressor(max_depth=15)
RMSE on validation data: 25.187563535165204
RMSE on train data: 12.619904226221944


RandomForestRegressor(max_depth=20)
RMSE on validation data: 25.37433295220108
RMSE on train data: 8.637916044618665


RandomForestRegressor(max_depth=25)
RMSE on validation data: 25.075394576897917
RMSE on train data

Vil jobbe videre med den beste modellen, Random Forest Regressor og endre på antall estimatorer og størrelse på dataset med mål om å forbedre den enda mer.

Videre bruker jeg Grid-Search fra denne kilden: https://medium.datadriveninvestor.com/random-forest-regression-9871bc9a25eb. Har endret på den slik at den kjører på samme måte som i run_model funksjonen ovenfor, men på testdataen.

Hyperparameterne som blir brukt i den endelige modellen vil ha de mest optimale verdiene, som velges i rfr_model().
Sklearn dokumentasjonen sier at max_depth er den maksimume dypden i treet og n_estimators antall trær i "forest".

I run_model() fant jeg ut at de beste verdiene var når max_depth var mellom 10 og 25. Utenfor disse mengdene opplevde jeg at resultatene ble dårligere og dårligere. Bruker derfor range(15,25).

In [16]:
def rfr_model(X_train, y_train, X_test, y_test, X_val, y_val):
# Perform Grid-Search
    gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(15,25),
            'n_estimators': (10, 50,100),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0,n_jobs=-1)
    
    grid_result = gsc.fit(X_train, y_train)
    best_params = grid_result.best_params_
    
    # lagrer modellen med de mest optimale verdiene, som blir brukt i "hyperparameterene", max_depth og n_estimators
    rfr = RandomForestRegressor(max_depth=best_params['max_depth'],
                                n_estimators=best_params['n_estimators'],
                                random_state=False, verbose=False)
    # fit model
    rfr.fit(X_train, y_train)
    
    # RMSE for validation data
    y_pred_val = model.predict(X_val)
    RMSE_val = mean_squared_error(y_val, y_pred_val, squared = False)
    
    # prediksjon av ny data (test data)
    y_pred =rfr.predict(X_test)

    # beregner root mean squared error 
    RMSE_test = mean_squared_error(y_test, y_pred, squared = False)
    
    print("\n")
    print(format(rfr))
    print("\n")
    print("RMSE on validation data:", RMSE_val)
    print("RMSE on test data:", RMSE_test)
    
    return rfr

model = rfr_model(X_train,y_train,X_val,y_val,X_test,y_test)



RandomForestRegressor(max_depth=20, random_state=False, verbose=False)


RMSE on validation data: 31.63093833113128
RMSE on test data: 25.368960940675493


In [17]:
# Predikerer Volum for 2022 
y_target = model.predict(target_df)
target_df['Volum'] = y_target
target_df

,År,Måned,Dato,Ukedag,Klokkeslett,Lufttemperatur,Lufttrykk,Globalstraling,Solskinstid,Fellesferie,Volum
0,2022.0,1.0,1.0,6.0,0.0,3.016667,1004.600000,-0.400000,0.000000,0.0,2.640000
1,2022.0,1.0,1.0,6.0,1.0,2.666667,1005.066667,-0.150000,0.000000,0.0,1.873333
2,2022.0,1.0,1.0,6.0,2.0,2.316667,1005.533333,-0.300000,0.000000,0.0,1.761667
3,2022.0,1.0,1.0,6.0,3.0,1.733333,1005.433333,-0.700000,0.000000,0.0,1.200917
4,2022.0,1.0,1.0,6.0,4.0,1.100000,1005.216667,-0.633333,0.000000,0.0,0.949603
...,...,...,...,...,...,...,...,...,...,...,...
3591,2022.0,5.0,30.0,1.0,16.0,6.001081,1005.888569,89.405867,0.934982,0.0,210.581338
3592,2022.0,5.0,30.0,1.0,17.0,6.001081,1005.888569,89.405867,0.934982,0.0,96.144500
3593,2022.0,5.0,30.0,1.0,18.0,6.001081,1005.888569,89.405867,0.934982,0.0,79.255000
3594,2022.0,5.0,30.0,1.0,19.0,6.001081,1005.888569,89.405867,0.934982,0.0,60.250000


In [18]:
# Plasser filene
target_df.to_csv('predicted_data/predictions.csv', index = False)

In [19]:
# Save model

pickle.dump({
    'model': model},
    open('model.pkl', 'wb'))